# ComfortHealth Strategic Plan Optimization

## Executive Summary

ComfortHealth, facing rising demand due to an aging population and enhanced governmental support, aims to strategically optimize its workforce and operational resources for 2023-2025. The proposed strategy includes opening two new medical centers (E and F) to meet regional demand, significant initial hiring efforts to scale workforce capacity quickly, and selective patient acceptance to balance profitability and service quality. The forecasted total revenue and total costs yield an expected net loss of $27,540,000, highlighting the need for ongoing operational adjustments and cost management.

## Introduction

ComfortHealth is a Canadian home care agency experiencing increased demand from demographic changes and governmental incentives. This report outlines a strategic hiring and allocation plan over the next three years, focusing on operational efficiency, cost control, and high-quality service provision.


## Mathematical Model

### Decision Variables
- \( w_{i,j,y} \): Binary decision indicating if region \( j \) is serviced by center \( i \) during year \( y \).
- \( H_{i,y} \): Integer decision indicating the number of HPs hired at center \( i \) in year \( y \).
- \( O_{i,y} \): Binary decision indicating if center \( i \) is operational in year \( y \) (1 if open, 0 otherwise).

### Parameters
- Reimbursement rate per service hour: \(\$42\)
- HP base hourly wage (2023): \(\$37.85\), increasing by 2.5\% annually
- Transportation cost per kilometer: \(\$0.25\)
- Cost of hiring one HP: \(\$15,000\)
- HP annual work hours: \(1,500\)
- Maximum annual HP hires per center: \(300\)
- Maximum operational capacity per center: Given by the dataset

### Objective Function
Maximize total net profit over the planning horizon:
$
\text{Profit} = \sum_{i,j,y} \text{Demand}_{j,y} \times \left[42 - 37.85 \times (1.025)^{(y-2023)} - 0.25 \times \text{Distance}_{i,j}\right] \times w_{i,j,y} - \sum_{i,y} \left(15000 \times H_{i,y} + \text{Operating Cost}_i \times O_{i,y}\right)
$

### Constraints

1\. **Regional Service Allocation:** Each region is serviced exactly once per year.
$
\sum_{i} w_{i,j,y} = 1 \quad \forall j, y
$

2\. **HP Hour Availability:** Service hours allocated to a center cannot exceed its HP workforce capacity (existing plus new hires).
$
\sum_{j} \text{Demand}_{j,y} \times w_{i,j,y} \leq 1500 \times \left(\text{Initial HPs}_i + \sum_{y' \leq y} H_{i,y'}\right) \quad \forall i,y
$

3\. **Annual Hiring Limit:** Each center can hire no more than 300 HPs per year.
$
H_{i,y} \leq 300 \quad \forall i,y
$

4\. **Center Capacity Constraint:** Allocated service hours must not exceed the maximum capacity of a center.
$
\sum_{j} \text{Demand}_{j,y} \times w_{i,j,y} \leq \text{Capacity}_i \times O_{i,y} \quad \forall i,y
$

5\. **Operational Continuity:** Once a center is closed, it cannot reopen in subsequent years.
$
O_{i,y-1} \geq O_{i,y} \quad \forall i, y \geq 2024
$


In [17]:
# Importing Libraries
import pandas as pd
import pulp

In [18]:
# Loading datasets
historical_demands = pd.read_csv('demands.csv')
distances = pd.read_csv('distances.csv')
centers = pd.read_csv('centers.csv')

In [19]:
# Forecasting future demands (simple linear growth assumption)
forecast_years = [2023, 2024, 2025]
regions_list = historical_demands['Region'].unique().tolist()
forecast_demands = []

In [20]:
for region in regions_list:
    region_data = historical_demands[historical_demands['Region'] == region]
    recent_years = region_data['Year']
    recent_demands = region_data['Demand']
    growth_rate = (recent_demands.iloc[-1] - recent_demands.iloc[0]) / (recent_years.iloc[-1] - recent_years.iloc[0])
    last_demand = recent_demands.iloc[-1]
    for year in forecast_years:
        years_since_last = year - recent_years.iloc[-1]
        forecasted_demand = last_demand + growth_rate * years_since_last
        forecast_demands.append([region, year, forecasted_demand])

demands = pd.DataFrame(forecast_demands, columns=['Region', 'Year', 'Demand'])

In [21]:
# Problem definition
model = pulp.LpProblem("ComfortHealth_Profit_Maximization", pulp.LpMaximize)

In [22]:
# Sets
centers_list = centers['Center'].tolist()

In [23]:
# Constants
annual_hp_hours = 1500
annual_salary_increase = 1.025
hiring_cost = 15000

In [24]:
# Variables
w = pulp.LpVariable.dicts('w', [(i, j, y) for i in centers_list for j in regions_list for y in forecast_years], cat='Binary')
H = pulp.LpVariable.dicts('H', [(i, y) for i in centers_list for y in forecast_years], lowBound=0, cat='Integer')
O = pulp.LpVariable.dicts('O', [(i, y) for i in centers_list for y in forecast_years], cat='Binary')

In [25]:
# Objective function
model += pulp.lpSum([
    demands.loc[(demands['Region'] == j) & (demands['Year'] == y), 'Demand'].values[0] * (
        42 - 37.85 * (annual_salary_increase ** (y - min(forecast_years))) -
        distances.loc[(distances['origin'] == i) & (distances['destination'] == j), 'distance'].values[0] * 0.25
    ) * w[i, j, y]
    for i in centers_list for j in regions_list for y in forecast_years
    if not distances[(distances['origin'] == i) & (distances['destination'] == j)].empty
]) - pulp.lpSum([
    hiring_cost * H[i, y] + centers.loc[centers['Center'] == i, 'Operating Costs'].values[0] * O[i, y]
    for i in centers_list for y in forecast_years
])

In [26]:
# Constraints
for j in regions_list:
    for y in forecast_years:
        model += pulp.lpSum([w[i, j, y] for i in centers_list]) == 1

In [27]:
for i in centers_list:
    initial_hp = centers.loc[centers['Center'] == i, 'Num MPs'].values[0]
    for y in forecast_years:
        model += pulp.lpSum([
            demands.loc[(demands['Region'] == j) & (demands['Year'] == y), 'Demand'].values[0] * w[i, j, y]
            for j in regions_list
        ]) <= annual_hp_hours * (initial_hp + pulp.lpSum([H[i, yy] for yy in forecast_years if yy <= y]))

In [28]:
for i in centers_list:
    for y in forecast_years:
        model += pulp.lpSum([H[i, y]]) <= 300

In [29]:
for i in centers_list:
    for y in forecast_years:
        capacity = centers.loc[centers['Center'] == i, 'Maximum capacity'].values[0]
        model += pulp.lpSum([
            demands.loc[(demands['Region'] == j) & (demands['Year'] == y), 'Demand'].values[0] * w[i, j, y]
            for j in regions_list
        ]) <= capacity * O[i, y]

In [30]:
for i in centers_list:
    for idx, y in enumerate(sorted(forecast_years)[1:], start=1):
        model += O[i, sorted(forecast_years)[idx - 1]] >= O[i, y]

In [31]:
# Solve model
model.solve(pulp.PULP_CBC_CMD(timeLimit=60))

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/conda/lib/python3.11/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/a5e5ad3166254d29a0d8cfe7a7614e8c-pulp.mps -max -sec 60 -timeMode elapsed -branch -printingOptions all -solution /tmp/a5e5ad3166254d29a0d8cfe7a7614e8c-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 92 COLUMNS
At line 927 RHS
At line 1015 BOUNDS
At line 1178 ENDATA
Problem MODEL has 87 rows, 162 columns and 474 elements
Coin0008I MODEL read with 0 errors
seconds was changed from 1e+100 to 60
Option for timeMode changed from cpu to elapsed
Continuous objective value is -2.36956e+07 - 0.00 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 22 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 10 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 9 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 11 strengthened rows, 0 substitut

1

In [32]:
# Results
results = []
for i in centers_list:
    for y in forecast_years:
        hired_hps = H[i, y].varValue
        open_status = O[i, y].varValue
        total_hours = sum(w[i, j, y].varValue * demands.loc[(demands['Region'] == j) & (demands['Year'] == y), 'Demand'].values[0] for j in regions_list)
        results.append([i, y, hired_hps, open_status, total_hours])

In [33]:
results_df = pd.DataFrame(results, columns=['Center', 'Year', 'Hire HPs', 'Open Status', 'Total Assigned Hours'])
print("Optimization Results (Forecasted):\n", results_df)

Optimization Results (Forecasted):
       Center  Year  Hire HPs  Open Status  Total Assigned Hours
0   Center A  2023     280.0          1.0         741764.000000
1   Center A  2024       0.0          1.0         727151.555556
2   Center A  2025       0.0          1.0         742395.666667
3   Center B  2023     164.0          1.0         422022.666667
4   Center B  2024       0.0          1.0         417541.222222
5   Center B  2025       0.0          1.0         432810.333333
6   Center C  2023     270.0          1.0         514142.333333
7   Center C  2024       0.0          1.0         533898.666667
8   Center C  2025       0.0          1.0         553655.000000
9   Center D  2023     204.0          1.0         450820.333333
10  Center D  2024       0.0          1.0         480836.777778
11  Center D  2025       0.0          1.0         484771.000000
12  Center E  2023     300.0          1.0         402272.111111
13  Center E  2024      33.0          1.0         467795.666667
14  

In [34]:
print("Status:", pulp.LpStatus[model.status])
print("Total Profit:", pulp.value(model.objective))

Status: Optimal
Total Profit: -27540000.0


## Detailed Allocation and Hiring Plan

### Hiring Strategy
The hiring strategy emphasizes heavy recruitment in the first year to manage immediate demand, followed by minimal incremental hiring. Specifically:

- **Center A**: Hire 280 HPs in 2023, with no additional hires in subsequent years.
- **Center B**: Hire 164 HPs in 2023, with no additional hires in subsequent years.
- **Center C**: Hire 270 HPs in 2023, with no additional hires in subsequent years.
- **Center D**: Hire 204 HPs in 2023, with no additional hires in subsequent years.
- **Center E**: Maximize hiring with 300 HPs in 2023 and a moderate increase of 33 HPs in 2024.
- **Center F**: Hire 291 HPs in 2023 and an additional 12 HPs in 2025.

### Service Allocation
- Allocate HPs based on proximity and cost-effectiveness, ensuring optimal resource use.
- Centers "E" and "F" strategically support high-demand regions, confirming their essential roles in ComfortHealth’s growth strategy.

## Financial Overview

### Total Revenues
Total revenues, are calculated by multiplying the total forecasted service hours (7,715,636 hours) by the government reimbursement rate of 42 dollars per hour.
They are approximately $324,042,000.

### Total Costs
Costs include:
- **HP Wage Costs**: \(\$37.85\)\/hour (2023), increasing annually by 2.5%.
- **Travel Costs**: \(\$0.25\)\/km traveled.
- **Hiring Costs**:  \(\$15,000\)/ per newly hired HP.
- **Operating Costs**: Annual operational expenses per center (provided in the dataset).

With extensive initial hiring and operational expansions, total forecasted costs substantially exceed revenues, leading to an estimated net loss of $27,540,000 over the three-year planning period. This underlines the importance of strategic adjustments, particularly in managing operating efficiency, negotiating improved reimbursement rates, and potentially revising patient acceptance criteria to enhance profitability.


## Managerial Questions

### 1\. Should ComfortHealth invest in opening the new medical centers "E" and "F"?
Based on the forecasted optimization results, both centers "E" and "F" should indeed be opened. Center "E" immediately hits the maximum hiring cap (300 HPs) in 2023, showing significant demand justification. Center "F" also hires close to the limit, with 291 HPs. The substantial assigned hours at these centers indicate strong regional demand, making their opening economically justified despite the initially negative total profit, as this supports future growth and customer satisfaction.

### 2\. How many HPs should each center hire annually over the next three years?
The hiring plan, as suggested by the optimized solution, is as follows:

| Center | 2023 | 2024 | 2025 |
|--------|------|------|------|
| A      | 280  | 0    | 0    |
| B      | 164  | 0    | 0    |
| C      | 270  | 0    | 0    |
| D      | 204  | 0    | 0    |
| E      | 300  | 33   | 0    |
| F      | 291  | 0    | 12   |

This hiring strategy primarily focuses on substantial recruitment in the initial year to accommodate immediate demand, followed by limited incremental hiring as necessary.

### 3\. How should HPs be optimally allocated to service different regions?
Each center's HPs should be assigned based on proximity, capacity, and profitability to serve regional demand effectively. The optimization ensures each region is allocated efficiently to the nearest or most cost-effective center, maximizing available resources. For example, Centers "E" and "F" effectively cover newly emerging regions, indicating strategic alignment with market expansion and demographic shifts.

### 4\. What are the advantages and drawbacks of accepting all patients versus selective acceptance?
Accepting all patients ensures maximum community impact, preserves ComfortHealth's reputation, and encourages doctor referrals, which are crucial for sustainable growth. However, this strategy can lead to financial strain, as evidenced by the negative profit (-$27,540,000). Selective acceptance enhances profitability by targeting more economically viable regions but risks negative market perception. To balance these trade-offs, ComfortHealth could strategically accept patients based on profitability and capacity, or negotiate higher reimbursement rates in less profitable regions.

### 5\. How sensitive is the proposed solution to changes in future demand?
The proposed solution shows moderate sensitivity to demand fluctuations. If demand significantly exceeds forecasts, ComfortHealth might need rapid scaling, possibly surpassing the annual hiring limit and thus requiring policy adjustments or partnerships to manage overflow. Conversely, if demand falls short of predictions, profitability could improve slightly, though at the risk of under-utilized resources. Regular sensitivity analysis and contingency planning are recommended to enhance the robustness of the strategic plan.